In [1]:
import torch

In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('protein-optimization/sc_diff/model-ylwt16su:v59', type='model')
artifact_dir = artifact.download()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: johnyang (protein-optimization). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-ylwt16su:v59, 96.20MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.2


In [3]:
artifact_dir

'./artifacts/model-ylwt16su:v59'

In [4]:
from cellot.models.cond_score_module import CondScoreModuleV2

In [5]:
ckpt_path = f'{artifact_dir}/model.ckpt'

In [6]:
YAML_STR = '''
DEBUG: False
TARGET: trametinib
LATENT_DIM: 50
COND_CLASSES: 190
SEED: 42
AE_PATH: /Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae
VAL_SIZE: 0.1
DEVICES: 1
WARM_START: False
WARM_START_PATH: null
MODEL_CLASS: CondScoreModule


diffuser:
  min_b: 0.01
  max_b: 1.0
  schedule: exponential
  score_scaling: var
  coordinate_scaling: 1.0
  latent_dim: ${LATENT_DIM}
  dt: 0.01
  min_t: 0

ae:
  name: scgen
  beta: 0.0
  dropout: 0.0
  hidden_units: [512, 512]
  latent_dim: 50

score_network:
  latent_dim: ${LATENT_DIM}
  cond_classes: ${COND_CLASSES}
  model_dim: 256   # Adjusted to 64
  n_layers: 6    # Adjusted to 12
  nhead: 8
  dim_feedforward: 2048
  dropout: 0.1
  ffn_hidden_dim: 1024
  extra_null_cond_embedding: False


data:
  type: cell
  source: control
  condition: drug
  path: /Mounts/rbg-storage1/users/johnyang/cellot/datasets/scrna-sciplex3/hvg.h5ad
  target: ${TARGET}

datasplit:
  groupby: drug   
  name: train_test
  test_size: 0.2
  random_state: 0
  
dataloader:
  batch_size: 256   # Adjusted to 256
  shuffle: true
  num_workers: 80
  
experiment:
  name: base
  mode: train
  num_loader_workers: 0
  port: 12319
  dist_mode: single
  use_wandb: True
  ckpt_path: null
  wandb_logger:
    project: sc_diff
    name: ${experiment.name}
    dir: /Mounts/rbg-storage1/users/johnyang/cellot/
    log_model: all
    tags: ['experimental']
  lr: 0.0001


trainer:
  accelerator: 'gpu'
  check_val_every_n_epoch: 50
  log_every_n_steps: 100
  num_sanity_val_steps: 1
  enable_progress_bar: True
  enable_checkpointing: True
  fast_dev_run: False
  profiler: simple
  max_epochs: 10000
  strategy: auto
  enable_model_summary: True
  overfit_batches: 0.0
  limit_train_batches: 1.0
  limit_val_batches: 1.0
  limit_predict_batches: 1.0
'''

In [7]:
from omegaconf import OmegaConf
config = OmegaConf.create(YAML_STR)

In [8]:
ckpt_path = f'{artifact_dir}/model.ckpt'

In [9]:
from cellot.train.utils import get_free_gpu
replica_id = int(get_free_gpu())

cuda
Using GPUs: 1


In [10]:
device = f'cuda:{replica_id}'

In [11]:
config

{'DEBUG': False, 'TARGET': 'trametinib', 'LATENT_DIM': 50, 'COND_CLASSES': 190, 'SEED': 42, 'AE_PATH': '/Mounts/rbg-storage1/users/johnyang/cellot/results/sciplex3/full_ae', 'VAL_SIZE': 0.1, 'DEVICES': 1, 'WARM_START': False, 'WARM_START_PATH': None, 'MODEL_CLASS': 'CondScoreModule', 'diffuser': {'min_b': 0.01, 'max_b': 1.0, 'schedule': 'exponential', 'score_scaling': 'var', 'coordinate_scaling': 1.0, 'latent_dim': '${LATENT_DIM}', 'dt': 0.01, 'min_t': 0}, 'ae': {'name': 'scgen', 'beta': 0.0, 'dropout': 0.0, 'hidden_units': [512, 512], 'latent_dim': 50}, 'score_network': {'latent_dim': '${LATENT_DIM}', 'cond_classes': '${COND_CLASSES}', 'model_dim': 256, 'n_layers': 6, 'nhead': 8, 'dim_feedforward': 2048, 'dropout': 0.1, 'ffn_hidden_dim': 1024, 'extra_null_cond_embedding': False}, 'data': {'type': 'cell', 'source': 'control', 'condition': 'drug', 'path': '/Mounts/rbg-storage1/users/johnyang/cellot/datasets/scrna-sciplex3/hvg.h5ad', 'target': '${TARGET}'}, 'datasplit': {'groupby': 'drug

In [12]:
lm = CondScoreModuleV2.load_from_checkpoint(hparams=config, checkpoint_path=ckpt_path).to(device)
print('')

Dropout is 0.1



In [13]:
# %%
import cellot.models
from cellot.data.cell import load_cell_data
import torch
from cellot.models.ae import AutoEncoder
from pathlib import Path

def load_data(config, **kwargs):
    data_type = config.get("data.type", "cell")
    if data_type in ["cell", "cell-merged", "tupro-cohort"]:
        loadfxn = load_cell_data

    elif data_type == "toy":
        loadfxn = load_toy_data

    else:
        raise ValueError

    return loadfxn(config, **kwargs)


def load_model(config, device, restore=None, **kwargs):
    # def load_autoencoder_model(config, restore=None, **kwargs):
    
    def load_optimizer(config, params):
        kwargs = dict(config.get("optim", {}))
        assert kwargs.pop("optimizer", "Adam") == "Adam"
        optim = torch.optim.Adam(params, **kwargs)
        return optim


    def load_networks(config, **kwargs):
        kwargs = kwargs.copy()
        kwargs.update(dict(config.get("ae", {})))
        name = kwargs.pop("name")

        # if name == "scgen":
        model = AutoEncoder

        # elif name == "cae":
        #     model = ConditionalAutoEncoder
        # else:
        #     raise ValueError

        return model(**kwargs)
    
    model = load_networks(config, **kwargs)
    optim = load_optimizer(config, model.parameters())

    if restore is not None and Path(restore).exists():
        print('Loading model from checkpoint')
        ckpt = torch.load(restore, map_location=device)
        model.load_state_dict(ckpt["model_state"])
        optim.load_state_dict(ckpt["optim_state"])
        # if config.model.name == "scgen" and "code_means" in ckpt:
        #     model.code_means = ckpt["code_means"]
            
    # logger.info(f'Model on device {next(model.parameters()).device}')

    return model, optim

def load(config, device, restore=None, include_model_kwargs=False, **kwargs):

    loader, model_kwargs = load_data(config, include_model_kwargs=True, **kwargs)

    model, opt = load_model(config, device, restore=restore, **model_kwargs)

    return model, opt, loader
# %% [markdown]
# ### Training

# %%
restore_path = '/Mounts/rbg-storage1/users/johnyang/cellot/saved_weights/ae/ae.pt'
ae = load_model(config, 'cuda', restore=restore_path, input_dim=1000)

Loading model from checkpoint


In [14]:
ae[0].decode

<bound method AutoEncoder.decode of AutoEncoder(
  (encoder_net): Sequential(
    (0): Linear(in_features=1000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=50, bias=True)
  )
  (decoder_net): Sequential(
    (0): Linear(in_features=50, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1000, bias=True)
  )
  (mse): MSELoss()
)>

In [15]:
import numpy as np

In [16]:
autoencoder = ae[0].to(device)

In [17]:
def inference(lm, batch, lamb=4, dt=0.01, t_start=1.0, cond=True):
    with torch.inference_mode():
        lm.eval()
        all_genes_x, y = batch
        latent_x = autoencoder.eval().encode(all_genes_x)
        
        x_t, _ = lm.diffuser.forward_marginal(latent_x.detach().cpu().numpy(), t=t_start)
        
        for i, t in enumerate(np.arange(t_start, 0, -dt)):
            x_t = torch.tensor(x_t).float().to(lm.device)
            uncond_score = lm.score_network((x_t, (torch.ones_like(y) * lm.score_network.null_cond_idx).to(device)), t)
            if cond:
                cond_score = lm.score_network((x_t, y), t)
                pred_score = (1 + lamb) * cond_score - lamb * uncond_score
            else:
                pred_score = uncond_score
            
            x_t = lm.diffuser.reverse(x_t=x_t.detach().cpu().numpy(), score_t=pred_score.detach().cpu().numpy(), t=t, dt=lm.dt, center=False)
        
        x_0 = torch.tensor(x_t, dtype=torch.float).to(lm.device)
        
        recon = autoencoder.eval().decode(x_0)
        return recon, latent_x, x_0
        
        

In [18]:
from cellot.utils.dev_utils import load_markers

sel_mg, gene_idxs = load_markers(config)
sel_mg

2023-07-10 21:03:13,059 Loaded cell data with TARGET trametinib and OBS SHAPE (20842, 16)


Index(['ENSG00000198074.9', 'ENSG00000019186.9', 'ENSG00000108846.15',
       'ENSG00000115414.18', 'ENSG00000231185.6', 'ENSG00000112541.13',
       'ENSG00000117983.17', 'ENSG00000145819.15', 'ENSG00000184588.17',
       'ENSG00000165376.10', 'ENSG00000154529.14', 'ENSG00000182752.9',
       'ENSG00000251003.7', 'ENSG00000101144.12', 'ENSG00000117724.12',
       'ENSG00000157168.18', 'ENSG00000275395.5', 'ENSG00000185483.11',
       'ENSG00000108405.3', 'ENSG00000089199.9', 'ENSG00000254166.2',
       'ENSG00000215182.8', 'ENSG00000004948.13', 'ENSG00000227706.3',
       'ENSG00000065809.13', 'ENSG00000004799.7', 'ENSG00000144847.12',
       'ENSG00000107957.16', 'ENSG00000108602.17', 'ENSG00000059804.15',
       'ENSG00000047648.21', 'ENSG00000076706.16', 'ENSG00000003436.15',
       'ENSG00000229140.8', 'ENSG00000066279.17', 'ENSG00000153956.15',
       'ENSG00000086548.8', 'ENSG00000171408.13', 'ENSG00000005108.15',
       'ENSG00000138696.10', 'ENSG00000236213.1', 'ENSG0000003842

In [20]:
from cellot.data.utils import *

In [21]:
def DEV_load_ae_cell_data(
        config,
        data=None,
        split_on=None,
        return_as="loader",
        include_model_kwargs=False,
        pair_batch_on=None,
        ae=None,
        encode_latents=False,
        sel_mg=None,
        **kwargs
    ):
        assert ae is not None or not encode_latents, "ae must be provided"
        
        if isinstance(return_as, str):
            return_as = [return_as]

        assert set(return_as).issubset({"anndata", "dataset", "loader"})
        config.data.condition = config.data.get("condition", "drug")
        condition = config.data.condition
        
        data = read_single_anndata(config, **kwargs)
        
        inputs = torch.Tensor(
            data.X if not sparse.issparse(data.X) else data.X.todense()
        )

        if encode_latents:
            genes = data.var_names.to_list()
            data = anndata.AnnData(
                ae.eval().encode(inputs).detach().numpy(),
                obs=data.obs.copy(),
                uns=data.uns.copy(),
            )
            data.uns["genes"] = genes


        # cast to dense and check for nans
        if sparse.issparse(data.X):
            data.X = data.X.todense()
        assert not np.isnan(data.X).any()

        if sel_mg is not None:
            data = data[:, sel_mg]

        dataset_args = dict()
        model_kwargs = {}

        model_kwargs["input_dim"] = data.n_vars

        # if config.get("model.name") == "cae":
        condition_labels = sorted(data.obs[condition].cat.categories)
        model_kwargs["conditions"] = condition_labels
        dataset_args["obs"] = condition
        dataset_args["categories"] = condition_labels

        if "training" in config:
            pair_batch_on = config.training.get("pair_batch_on", pair_batch_on)

        # if split_on is None:
            # if config.model.name == "cellot":
            #     # datasets & dataloaders accessed as loader.train.source
        split_on = ["split", "transport"]
        if pair_batch_on is not None:
            split_on.append(pair_batch_on)

            # if (config.ae.name == "scgen" #or config.ae.name == "cae"
            #     #or config.ae.name == "popalign"):
            # split_on = ["split"]

            # else:
            #     raise ValueError

        if isinstance(split_on, str):
            split_on = [split_on]

        for key in split_on:
            assert key in data.obs.columns

        if len(split_on) > 0:
            splits = {
                (key if isinstance(key, str) else ".".join(key)): data[index]
                for key, index in data.obs[split_on].groupby(split_on).groups.items()
            }

            dataset = nest_dict(
                {
                    key: AnnDataDataset(val.copy(), **dataset_args)
                    for key, val in splits.items()
                },
                as_dot_dict=True,
            )
        else:
            dataset = AnnDataDataset(data.copy(), **dataset_args)

        if "loader" in return_as:
            kwargs = dict(config.dataloader)
            kwargs.setdefault("drop_last", True)
            loader = cast_dataset_to_loader(dataset, **kwargs)

        returns = list()
        for key in return_as:
            if key == "anndata":
                returns.append(data)

            elif key == "dataset":
                returns.append(dataset)

            elif key == "loader":
                returns.append(loader)

        if include_model_kwargs:
            returns.append(model_kwargs)

        if len(returns) == 1:
            return returns[0]

        # returns.append(data)

        return tuple(returns)

In [22]:
datasets = DEV_load_ae_cell_data(config, return_as='dataset')#, ae=autoencoder.cpu(), encode_latents=True)#, sel_mg=sel_mg)

In [23]:
loader = cast_dataset_to_loader(datasets, batch_size=256, shuffle=False, drop_last=False)
loader

{'test': {'source': <torch.utils.data.dataloader.DataLoader at 0x7f74d08ac700>,
  'target': <torch.utils.data.dataloader.DataLoader at 0x7f74d08ac820>},
 'train': {'source': <torch.utils.data.dataloader.DataLoader at 0x7f74d08ac280>,
  'target': <torch.utils.data.dataloader.DataLoader at 0x7f74d08aca30>}}

In [24]:
source = datasets.test.source.adata.X

In [25]:
target = datasets.test.target.adata.X

In [26]:
source[:, gene_idxs[:50]].shape

(3513, 50)

In [27]:
target[:, gene_idxs[:50]].shape

(656, 50)

In [28]:
from tqdm import tqdm

In [29]:
from cellot.losses.mmd import mmd_distance
import numpy as np

def compute_mmd_loss(lhs, rhs, gammas):
    return np.mean([mmd_distance(lhs, rhs, g) for g in gammas])

gammas = np.logspace(1, -3, num=50)

In [41]:
sel_target = target[:, gene_idxs[:50]]

In [42]:
sel_target

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.80774903, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 2.6273441 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [31]:
gts = []
recons = []
lxs = []
xs = []
for batch in tqdm(loader.test.source):
    batch = [x.to(device) for x in batch]
    gts.append(batch)
    recon, latent_x, recon_x_0 = inference(lm, batch, lamb=4, dt=0.01, t_start=1.0, cond=True)
    recons.append(recon)
    lxs.append(latent_x)
    xs.append(recon_x_0)
    

  0%|          | 0/14 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:19<00:00,  1.36s/it]


In [32]:
torch.where(gts[0][0][0, gene_idxs[:50]] > 0, 1, 0)

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0], device='cuda:1')

In [33]:
torch.where(recons[0][0, gene_idxs[:50]] > 0.1, 1, 0)

tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
        1, 1], device='cuda:1')

In [34]:
all_recon = torch.cat(recons, dim=0)
all_recon.shape

torch.Size([3513, 1000])

In [35]:
all_lxs = torch.cat(lxs, dim=0)
all_xs = torch.cat(xs, dim=0)

In [36]:
lx_targets = []
with torch.inference_mode():
    for batch in tqdm(loader.test.target):
        batch = [x.to(device) for x in batch]
        autoencoder.eval()
        x, y = batch
        latent_x_target = autoencoder.encode(x)
        lx_targets.append(latent_x_target)

100%|██████████| 3/3 [00:00<00:00, 92.57it/s]


In [37]:
all_lx_targets = torch.cat(lx_targets, dim=0)

In [38]:
compute_mmd_loss(all_lxs.cpu().numpy(), all_xs.cpu().numpy(), gammas)

0.0003377005992064311

In [39]:
compute_mmd_loss(all_xs.cpu().numpy(), all_lx_targets.cpu().numpy(), gammas)

0.005241116983967545

In [40]:
compute_mmd_loss(all_recon[:, gene_idxs[:50]].detach().cpu().numpy(), sel_target, gammas)

0.017621534580830486

In [67]:
compute_mmd_loss(all_recon[:, gene_idxs[:50]].detach().cpu().numpy(), sel_target, gammas)

0.03237136445939541

In [ ]:
compute_mmd_loss(all_recon[:, gene_idxs[:50]].detach().cpu().numpy(), sel_target, gammas)

In [43]:
from cellot import losses
losses.compute_scalar_mmd(all_recon[:, gene_idxs[:50]].detach().cpu().numpy(), sel_target)

0.0322643518447876